## Importing PWE and other essentials

In [1]:
from PW_explorer.run_clingo import run_clingo
from PW_explorer.load_worlds import load_worlds
from PW_explorer.pwe_helper import rel_slicer
from PW_explorer.time_series import PWETimeSeriesModule
from PW_explorer.pwe_nb_helper import ASPRules

In [2]:
%load_ext PWE_NB_Extension

## Formulating the Towers of Hanoi Problem

This is the towers of Hanoi instance with 3 pegs and 3 disks.

In [3]:
%%clingo --donot-run -lci toh_instance --donot-display_input

#const num_disks=3.
% schema peg(PEG)
peg(a;b;c).              % Define the 3 pegs a,b,c
% schema disk(DISK)
disk(1..num_disks).      % Define the three disks.
% schema init_on(DISK, PEG)
init_on(1..num_disks,a). % Define the initial position of the disks
% schema goal_on(DISK, PEG)
goal_on(1..num_disks,c). % Define the desired final position of the disks
moves(7).                % Define the maximum number of moves allowed

This is the encoding of the rules of the game

In [4]:
%%clingo --donot-run -lci toh_encoding --donot-display_input

% schema move(TIME, DISK, PEG)
% temporal move(T,_,_)
% Generate a move at each time-step
{ move(T,D,P) : disk(D), peg(P) } = 1 :- moves(M),  T = 1..M.

% schema move(TIME, DISK)
% temporal move(T,_)
move(T,D)   :- move(T,D,_).

% schema on(TIME, DISK, PEG)
% temporal on(T,_,_)
on(0,D,P)   :- init_on(D,P).

% Update state
% Update location of moved disk
on(T,D,P)   :- move(T,D,P).
% Maintain disk D’s position at time-step T+1 if not moved
on(T+1,D,P) :- on(T,D,P), not move(T+1,D), not moves(T).

% schema blocked(TIME, DISK, PEG)
% temporal blocked(T,_,_)

% If disk D is on peg P at time-step T, then at time-step T+1
% disk D-1 cannot be moved to peg P
blocked(T+1,D-1,P) :- on(T,D,P), not moves(T).
% If disk D cannot be moved to peg P at time-step T,
% then neither can disk D-1 (and so on..)
blocked(T,D-1,P)   :- blocked(T,D,P), disk(D).

% Test the validity of move at time-step T
:- move(T,D,P), blocked(T,D-1,P).
:- move(T,D), on(T-1,D,P), blocked(T,D,P).

% Test the validity of the sequence of moves:
% Series of steps doesn’t achieve goal
:- goal_on(D,P), not on(M,D,P), moves(M).
% Disk is not on exactly one peg at any time-step
:- { on(T,D,P) } != 1, disk(D), moves(M), T = 1..M.

%% % Display
%% #show move/3.

In [5]:
clingo_rules = toh_encoding.splitlines()+toh_instance.splitlines()

Running this carefully constructed to yield exactly one PW example with clingo 

In [6]:
clingo_soln, meta_data = run_clingo(clingo_rules=clingo_rules)
ASPRules('\n'.join(clingo_soln))

'Answer: 1\ndisk(1) disk(2) disk(3) peg(a) peg(b) peg(c) moves(7) init_on(1,a) init_on(2,a) init_on(3,a) on(0,1,a) on(0,2,a) on(0,3,a) blocked(1,0,a) blocked(1,1,a) blocked(1,2,a) goal_on(1,c) goal_on(2,c) goal_on(3,c) on(1,1,a) on(1,2,a) blocked(2,0,a) blocked(2,1,a) on(1,3,c) move(1,3,c) on(2,1,a) on(2,2,b) move(2,2,b) on(2,3,c) on(3,1,a) on(3,2,b) on(3,3,b) move(3,3,b) on(4,1,c) move(4,1,c) on(4,2,b) on(4,3,b) on(5,1,c) on(5,2,b) on(5,3,a) move(5,3,a) on(6,1,c) on(6,2,c) move(6,2,c) on(6,3,a) on(7,1,c) on(7,2,c) on(7,3,c) move(7,3,c) move(1,3) move(2,2) move(3,3) move(4,1) move(5,3) move(6,2) move(7,3) blocked(2,0,c) blocked(2,1,c) blocked(2,2,c) blocked(3,0,a) blocked(3,0,b) blocked(3,0,c) blocked(3,1,b) blocked(3,1,c) blocked(3,2,c) blocked(4,0,a) blocked(4,0,b) blocked(4,1,b) blocked(4,2,b) blocked(5,0,b) blocked(5,0,c) blocked(5,1,b) blocked(5,2,b) blocked(6,0,a) blocked(6,0,b) blocked(6,0,c) blocked(6,1,a) blocked(6,1,b) blocked(6,2,a) blocked(7,0,a) blocked(7,0,c) blocked(7,1,a) blocked(7,1,c) blocked(7,2,a)\nSATISFIABLE\n\nModels       : 1\nCalls        : 1\nTime         : 0.005s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)\nCPU Time     : 0.004s'

This is the extracted meta data for the ASP formulation

In [7]:
meta_data

{'temporal_dec': {'move_3': [0], 'move_2': [0], 'on_3': [0], 'blocked_3': [0]},
 'attr_def': {'move_3': ['TIME', 'DISK', 'PEG'],
  'move_2': ['TIME', 'DISK'],
  'on_3': ['TIME', 'DISK', 'PEG'],
  'blocked_3': ['TIME', 'DISK', 'PEG'],
  'peg_1': ['PEG'],
  'disk_1': ['DISK'],
  'init_on_2': ['DISK', 'PEG'],
  'goal_on_2': ['DISK', 'PEG']},
 'graphviz': {'graph': {'graph_type': 'undirected', 'styles': []},
  'node': {},
  'edge': {}}}

Now we load the solution using PWE

In [8]:
pw_rels_dfs, rel_schemas, pw_objs = load_worlds(asp_output=clingo_soln, meta_data=meta_data, reasoner='clingo')

Number of Models: 1


In [9]:
list(map(lambda x: x.__dict__, rel_schemas))

[{'relation_name': 'disk_1',
  'arity': 1,
  'r_id': 0,
  'meta_data': {'attr_def': ['DISK']}},
 {'relation_name': 'peg_1',
  'arity': 1,
  'r_id': 1,
  'meta_data': {'attr_def': ['PEG']}},
 {'relation_name': 'moves_1', 'arity': 1, 'r_id': 2, 'meta_data': {}},
 {'relation_name': 'init_on_2',
  'arity': 2,
  'r_id': 3,
  'meta_data': {'attr_def': ['DISK', 'PEG']}},
 {'relation_name': 'on_3',
  'arity': 3,
  'r_id': 4,
  'meta_data': {'attr_def': ['TIME', 'DISK', 'PEG'], 'temporal_dec': [0]}},
 {'relation_name': 'blocked_3',
  'arity': 3,
  'r_id': 5,
  'meta_data': {'attr_def': ['TIME', 'DISK', 'PEG'], 'temporal_dec': [0]}},
 {'relation_name': 'goal_on_2',
  'arity': 2,
  'r_id': 6,
  'meta_data': {'attr_def': ['DISK', 'PEG']}},
 {'relation_name': 'move_3',
  'arity': 3,
  'r_id': 7,
  'meta_data': {'attr_def': ['TIME', 'DISK', 'PEG'], 'temporal_dec': [0]}},
 {'relation_name': 'move_2',
  'arity': 2,
  'r_id': 8,
  'meta_data': {'attr_def': ['TIME', 'DISK'], 'temporal_dec': [0]}}]

We have parsed the following relations

In [10]:
pw_rels_dfs.keys()

dict_keys(['disk_1', 'peg_1', 'moves_1', 'init_on_2', 'on_3', 'blocked_3', 'goal_on_2', 'move_3', 'move_2'])

And they are each stored in a Pandas Dataframe which looks like this:

(Also notice that the column names were inferred from the meta_data we parsed earlier)

In [11]:
pw_rels_dfs['on_3']

,pw,TIME,DISK,PEG
0,1,0,1,a
1,1,0,2,a
2,1,0,3,a
3,1,1,1,a
4,1,1,2,a
5,1,1,3,c
6,1,2,1,a
7,1,2,2,b
8,1,2,3,c
9,1,3,1,a


##### All these are hard to make sense of, especially since there's time states involved.
##### A simple groupby on the temporal columns would go a long way

In [12]:
sliced_dfs, sliced_rels, _ = rel_slicer(pw_rels_dfs, rel_schemas, None, ['move_3', 'on_3', 
                                                                         'init_on_2', 'peg_1', 
                                                                         'goal_on_2', 'disk_1',])
timestep_state_map = PWETimeSeriesModule.group_by_time(sliced_dfs, sliced_rels)

In [13]:
timestep_state_map

{'constant': {'init_on_2':    pw DISK PEG
  0   1    1   a
  1   1    2   a
  2   1    3   a, 'disk_1':    pw DISK
  0   1    1
  1   1    2
  2   1    3, 'goal_on_2':    pw DISK PEG
  0   1    1   c
  1   1    2   c
  2   1    3   c, 'peg_1':    pw PEG
  0   1   a
  1   1   b
  2   1   c}, 0: {'on_3':    pw DISK PEG
  0   1    1   a
  1   1    2   a
  2   1    3   a}, 1: {'on_3':    pw DISK PEG
  3   1    1   a
  4   1    2   a
  5   1    3   c, 'move_3':    pw DISK PEG
  0   1    3   c}, 2: {'on_3':    pw DISK PEG
  6   1    1   a
  7   1    2   b
  8   1    3   c, 'move_3':    pw DISK PEG
  1   1    2   b}, 3: {'on_3':     pw DISK PEG
  9    1    1   a
  10   1    2   b
  11   1    3   b, 'move_3':    pw DISK PEG
  2   1    3   b}, 4: {'on_3':     pw DISK PEG
  12   1    1   c
  13   1    2   b
  14   1    3   b, 'move_3':    pw DISK PEG
  3   1    1   c}, 5: {'on_3':     pw DISK PEG
  15   1    1   c
  16   1    2   b
  17   1    3   a, 'move_3':    pw DISK PEG
  4   1    3   a}, 6

Not very fun to look at either

#### So we can use this built-in text-based visualization for a clearer understanding of the solution

In [14]:
PWETimeSeriesModule.simple_timeseries_text_visualization(timestep_state_map, jupyter=True)

Constants:

init_on_2:


,pw,DISK,PEG
0,1,1,a
1,1,2,a
2,1,3,a




disk_1:


,pw,DISK
0,1,1
1,1,2
2,1,3




goal_on_2:


...




---------------
Timestep 7:
---------------

on_3:


,pw,DISK,PEG
21,1,1,c
22,1,2,c
23,1,3,c




move_3:


,pw,DISK,PEG
6,1,3,c



END


### Building a Custom Visualization for the TOH problem

In [15]:
# More informative custom visualization (still a WIP)
# Inspired by: https://stackoverflow.com/questions/49391076/illustrate-tower-of-hanoi-with-ascii
def toh_viz(timestep_state_map):
    
    NUM_DISKS = len(timestep_state_map['constant']['disk_1'])
    NUM_PEGS = len(timestep_state_map['constant']['peg_1'])
    PEG_SIZE = NUM_DISKS + 2
    
    def get_state(state_df):
        pegs = [[] for _ in range(NUM_PEGS)]
        for i, row in state_df.iterrows():
            pegs[ord(row['PEG']) - ord('a')].append(NUM_DISKS+1-int(row['DISK']))
        pegs = [sorted(peg) for peg in pegs]
        return pegs
    
    def render_ring(ring):
        result = '*' * ring  # the character * repeated ring times.
        return result.center(PEG_SIZE) # add the spaces required
    def render_tower(tower, tower_id):
        result = [render_ring(0) for _ in range(NUM_DISKS-len(tower))] #[]
        for ring in tower:
            result.append(render_ring(ring))
        result.append(str(tower_id).center(PEG_SIZE))
        return result
    def render_final(towers): 
        tower_results = []
        for i, tower in enumerate(towers):
            tower_results.append(render_tower(tower, chr(ord('a')+i)))
        result = []
        for all_rows in zip(*tower_results):
            result.append(''.join(all_rows)) 
        return result
    
    print("Initial State:")
    print('\n'.join(render_final(get_state(timestep_state_map['constant']['init_on_2']))))
    print("")
    print("Goal:")
    print('\n'.join(render_final(get_state(timestep_state_map['constant']['goal_on_2']))))
    print("")
    for t in sorted(set(timestep_state_map.keys()) - set(['constant'])):
        print("Timestep: {}".format(t))
        print("")
        if 'move_3' in timestep_state_map[t]:
            for i, row in timestep_state_map[t]['move_3'].iterrows():
                print("Move {} to peg {}.".format('*'*(NUM_DISKS+1-int(row['DISK'])), row['PEG']))
        print('\n'.join(render_final(get_state(timestep_state_map[t]['on_3']))))
        print("")
    

##### Now let's use the above user-defined visualization

In [16]:
toh_viz(timestep_state_map)

Initial State:
  *            
  **           
 ***           
  a    b    c  

Goal:
            *  
            ** 
           *** 
  a    b    c  

Timestep: 0

  *            
  **           
 ***           
  a    b    c  

Timestep: 1

Move * to peg c.
               
  **           
 ***        *  
  a    b    c  

Timestep: 2

Move ** to peg b.
               
               
 ***   **   *  
  a    b    c  

Timestep: 3

Move * to peg b.
               
       *       
 ***   **      
  a    b    c  

Timestep: 4

Move *** to peg c.
               
       *       
       **  *** 
  a    b    c  

Timestep: 5

Move * to peg a.
               
               
  *    **  *** 
  a    b    c  

Timestep: 6

Move ** to peg c.
               
            ** 
  *        *** 
  a    b    c  

Timestep: 7

Move * to peg c.
            *  
            ** 
           *** 
  a    b    c  

